In [11]:
# ==============================================================================
# SCRIPT HEADER
# ==============================================================================
#
# Author:       [Umut TANK]
# Date Created: [April 2025]
# Contact:      [umuttank@hotmail.com]
# Project:      [Annotation of Proteomics Results for my Project]
#
# ==============================================================================
# SCRIPT DESCRIPTION
# ==============================================================================
#
# **Overall Purpose:**
# This script automates the process of fetching and integrating biological annotations
# for lists of proteins/genes from experimental results (e.g., proteomics analysis).
# It reads data from specified sheets within an input Excel file. It checks for a
# 'Significant' column; if present, it filters for significant entries ('+'), otherwise
# it processes all rows containing gene names. It retrieves functional information
# from various online databases, flags potential background contaminants, and
# outputs a new, richly annotated Excel file.
#
# **Workflow:**
# 1.  **Configuration:** Reads settings for file paths, API parameters (retries,
#     timeouts), analysis thresholds, background keywords, and specific column names.
# 2.  **Input Loading:** Loads the specified input Excel file (`.xlsx`).
# 3.  **Pfam Data Loading:** Loads Pfam domain-to-clan mapping information from a local TSV file.
# 4.  **Sheet Iteration:** Processes each sheet within the input Excel file sequentially.
# 5.  **Row Selection (Conditional):**
#     *   Checks if a 'Significant' column exists.
#     *   If YES: Filters rows marked as significant ('+') for annotation.
#     *   If NO: Selects all rows containing a non-empty 'Gene names' entry for annotation.
# 6.  **Gene Symbol Extraction:** Extracts the primary gene symbol from the relevant column
#     in the selected rows (handling potentially semicolon-separated lists).
# 7.  **Annotation Fetching (for Unique Selected Genes):** For each unique gene symbol:
#     *   Uses a caching mechanism to avoid redundant API calls for the same gene.
#     *   Fetches data from UniProt (protein description, function, location, keywords,
#       cross-references including Pfam, etc. **Note: UniProt GO terms are NOT fetched directly**).
#     *   Fetches interacting partners from STRING DB (physical and functional).
#     *   Fetches functional enrichment terms from g:Profiler (GO, pathways, complexes).
#     *   Fetches detailed GO terms (BP, MF, CC) from QuickGO.
#     *   Applies configured retries and backoff delays for robustness against API issues.
# 8.  **Data Processing & Integration:**
#     *   Parses the fetched data (e.g., extracts specific fields, looks up Pfam descriptions/clans).
#     *   Handles dynamic UniProt keyword categorization.
# 9.  **Mapping Annotations:** Maps the retrieved and processed annotations back to the
#     corresponding selected rows in the sheet's DataFrame.
# 10. **Background Flagging:** Applies flags based on:
#     *   Keywords in protein descriptions ('Sticky Binder Flag').
#     *   Expression patterns in EV control columns ('EV LFQ Background Flag').
#     *   Significance in a separate control sheet ('EV Control Background Flag').
# 11. **QC Check:** Compares input protein names to UniProt descriptions ('Name Match Discrepancy').
# 12. **Column Reordering:** Arranges the columns into a logical order, inserting annotations
#     after the 'Gene names' column.
# 13. **Output Writing:** Writes the annotated DataFrame for the current sheet to the
#     output Excel file using a safe writing function (handles sheet name cleaning).
#     *Note: Currently configured to save the entire workbook after each sheet is written.*
# 14. **Logging:** Records progress, warnings, errors, configuration details, and timing
#     to both the console and a timestamped log file.
#
# **Input:**
# *   **Primary:** An Excel file (`.xlsx`) containing one or more sheets with protein/gene data.
#     Expected columns: "Gene names". Optionally: "Significant", "Protein names",
#     "log2FC", and EV intensity columns (e.g., "EV_01").
# *   **Secondary (Optional but Recommended):** A local Pfam Clan TSV file (e.g., `Pfam-A.clans.tsv.gz`)
#     for mapping Pfam domains to descriptions and clans.
#
# **Output:**
# *   **Primary:** A new Excel file (`.xlsx`) named based on the input file plus "_Annotated_YYYYMMDD.xlsx",
#     containing the annotated data for the selected rows from each processed sheet.
# *   **Secondary:** A log file (`.log`) named `annotation_log_YYYYMMDD.log` containing detailed
#     information about the script's execution.
#
# **Key Features & Annotations Added:**
# *   **UniProt:** ID, Description, Function, Subcellular Location, Molecular Mass (kDa),
#     Alternative Names, Catalytic Activity, Associated Diseases, Sequence Similarities,
#     Pathways, Protein Existence Level, Categorized Keywords.
# *   **Pfam (via UniProt & Lookup):** PFAM Domain IDs, PFAM Descriptions, PFAM Clan Names.
# *   **STRING DB:** High-confidence Physical Interactors, Functional Partners.
# *   **g:Profiler:** Functional enrichment terms (GO:BP, GO:MF, GO:CC, REAC, KEGG, WP, CORUM).
# *   **QuickGO:** Detailed GO terms (BP, MF, CC) fetched via QuickGO API.
# *   **Flags:** Sticky Binder, EV LFQ Background, EV Control Background, Name Match QC.
# *   **Utilities:** API request caching, robust API retries with backoff, detailed logging,
#     conditional row selection based on 'Significant' column presence.
#
# **Configuration:**
# Key parameters like input/output paths, API settings, background keywords, column names,
# and analysis thresholds can be adjusted in the 'CONFIGURATION SETTINGS' (Section 3)
# and 'COLUMN NAME CONSTANTS' (Section 4) sections of the script.
#
# ==============================================================================
# (Script Code Starts Below)
# ==============================================================================

# === 1. INSTALL LIBRARIES (Run once if needed) ===
# Description: This section lists the necessary Python libraries.
#              Uncomment and run the '!pip install' line in your environment
#              (like a Jupyter notebook or terminal) if these libraries are not yet installed.
# !pip install requests openpyxl tqdm pandas gprofiler-official

# === 2. IMPORT REQUIRED LIBRARIES ===
# Description: Imports all the necessary libraries for the script's functionality.
#              These include file handling (os), time delays (time), data manipulation (pandas, numpy),
#              web requests (requests), progress bars (tqdm), gene enrichment (gprofiler),
#              robust web request handling (HTTPAdapter, Retry), URL encoding (quote),
#              date/time operations (datetime), logging (logging), type hints (typing),
#              and regular expressions (re).
import os
import time
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm  # For progress bars
from gprofiler import GProfiler # For gene set enrichment analysis
from requests.adapters import HTTPAdapter # For configuring advanced request options
from urllib3.util.retry import Retry # For automatic retries on failed requests
from urllib.parse import quote # For URL encoding gene symbols if needed (not directly used here but good practice)
# import warnings # No longer strictly needed as warnings.warn replaced by logging
from datetime import datetime # For timestamping output files and logs
import logging  # Added for structured logging of script progress and errors
import typing  # Added for type hinting (improves code readability and maintainability)
import re # Added for regex operations (e.g., cleaning sheet names, keyword checks)

# === 3. CONFIGURATION SETTINGS ===
# Description: Centralized configuration for file paths, API parameters, and analysis settings.
#              Modify these values according to your environment and analysis needs.

# --- User Configuration ---
# !! IMPORTANT: Set these paths correctly for your system !!
# INPUT_EXCEL_PATH: Path to the input Excel file containing protein/gene lists.
#                   Consider using command-line arguments or relative paths for better portability.
INPUT_EXCEL_PATH = "/Users/umuttank/Downloads/Results.xlsx"
# PFAM_CLAN_FILE_PATH: Path to the Pfam Clan TSV file (compressed or uncompressed).
#                      Download from Pfam FTP: ftp://ftp.ebi.ac.uk/pub/databases/Pfam/current_release/Pfam-A.clans.tsv.gz
PFAM_CLAN_FILE_PATH = "Pfam-A.clans.tsv.gz"
# --- End User Configuration ---

# --- Dynamic Output Path ---
# Generates the output file name based on the input name and the current date.
today = datetime.today().strftime("%Y%m%d")
output_path = INPUT_EXCEL_PATH.replace(".xlsx", f"_Annotated_{today}.xlsx")

# --- API and Analysis Parameters ---
CONFIG = {
    # UniProt API request settings (also used for STRING, QuickGO via the session)
    "uniprot": {
        "max_retries": 5,  # Max number of retry attempts (Total attempts = 1 + max_retries)
        "delay": 0.4,      # Backoff factor (wait_time = delay * (2 ** num_retries))
        "timeout": 20      # Seconds to wait for a response before timing out.
    },
    # g:Profiler enrichment analysis settings
    "gprofiler": {
        "sources": ["GO:BP", "GO:MF", "GO:CC", "REAC", "KEGG", "WP", "CORUM"], # Databases to query.
        "threshold": 0.05,         # Significance threshold (p-value) for enrichment terms.
        "max_terms": 5             # Maximum number of enriched terms to report per source per gene.
    },
    # Keywords used to flag potential "sticky" or common background proteins.
    "background_keywords": [
        "ribosomal", "histone", "tubulin", "actin", "myosin", "keratin",
        "elongation factor", "collagen", "fibrinogen",
        "HSP", "heat shock protein", "GAPDH", "aldolase", "annexin", "vimentin"
    ],
    # Column names in the input sheet used for EV LFQ background detection. Adjust if your names differ.
    "ev_columns": ["EV_01", "EV_02", "EV_03", "EV_04"],
    # Name of the sheet used to determine a list of general EV control background proteins.
    "ev_sheet_name": "EV vs EV_ISD",
    # "ev_threshold_multiplier": 2 # This key is defined but not used, can be removed if not needed
}

# === 4. COLUMN NAME CONSTANTS ===
# Description: Defines constants for column names used throughout the script.
#              This makes the code more readable and easier to maintain if column names change.

# --- Input Columns (Expected in the input Excel file) ---
COL_GENE_NAMES = "Gene names"         # Essential: Column containing gene symbols.
COL_PROTEIN_NAMES = "Protein names"   # Optional: Column containing protein names (used for QC check).
COL_SIGNIFICANT = "Significant"       # Optional: Column indicating significance (e.g., '+' for significant). If absent, all genes are processed.
COL_LOG2FC = "log2FC"             # Optional: Column containing log2 Fold Change values (used for EV LFQ flag).

# --- Added Columns (Generated by this script) ---
COL_MAIN_GENE = "Main Gene"           # The primary gene symbol extracted from COL_GENE_NAMES.

# Basic UniProt Annotations (fetched via UniProt REST API)
COL_UNIPROT_ID = "UniProt ID"             # Primary UniProt accession number.
COL_PROTEIN_DESC = "Protein Description"    # UniProt recommended full protein name.
COL_FUNCTION = "Function"               # Functional annotation from UniProt comments.
COL_LOCATION = "Subcellular Location"   # Subcellular location from UniProt comments.
COL_CANONICAL = "Canonical Isoform"     # Heuristic check if the UniProt ID represents the canonical sequence.
COL_MASS_KDA = "Molecular Mass (kDa)"   # Protein molecular weight calculated from sequence.

# Pfam Annotations (derived from UniProt cross-references and Pfam clan file)
COL_PFAM_DOMAINS = "PFAM Domains"           # Pfam domain IDs associated with the protein.
COL_PFAM_DESC = "PFAM Descriptions"       # Descriptions of the associated Pfam domains.
COL_PFAM_CLANS = "PFAM Clan Names"        # Names of Pfam clans the domains belong to.

# Extended UniProt Annotations (additional fields extracted from UniProt entry)
COL_ALT_NAMES_V2 = "Alternative Names"       # Alternative protein names from UniProt.
COL_CAT_ACT_V2 = "Catalytic Activity"      # Catalytic activity details from UniProt comments.
COL_DISEASES_V2 = "Associated Diseases"    # Disease associations from UniProt comments.
COL_SIMILARITIES_V2 = "Sequence Similarities" # Sequence similarity details from UniProt comments.
COL_PATHWAYS_V2 = "UniProt Pathways"       # Pathway information from UniProt comments.
COL_EXISTENCE_V2 = "Protein Existence Level" # UniProt evidence code for protein existence.

# Dynamically Generated UniProt Keyword Columns
# Example: "UniProt KW: Molecular_function", "UniProt KW: Biological_process"

# <<< REMOVED UniProt GO Term Columns >>>
# COL_UNIPROT_GO_CC = "UniProt GO:CC Terms"

# QuickGO Annotations (Fetched separately from QuickGO API)
# Columns added dynamically in main(): "QuickGO:BP Terms", "QuickGO:MF Terms", "QuickGO:CC Terms"

# STRING Annotations (Fetched from STRING DB API)
COL_STRING_PHYS = "STRING Physical Interactors" # High-confidence physical interactors.
COL_STRING_FUNC = "STRING Functional Partners"  # High-confidence functional partners.

# g:Profiler Annotations (Column names are generated dynamically based on CONFIG["gprofiler"]["sources"])
# Example: "GO:BP Terms", "REAC Terms", etc.

# Flag/QC Columns (Generated based on annotation data or input values)
COL_STICKY_FLAG = "Sticky Binder Flag"        # Flagged if protein description contains background keywords.
COL_EV_LFQ_FLAG = "EV LFQ Background Flag"    # Flagged based on EV intensities and log2FC criteria.
COL_EV_BG_FLAG = "EV Control Background Flag" # Flagged if gene is significant in the specified EV control sheet.
COL_NAME_MATCH = "Name Match Discrepancy"     # QC check: Flags potential mismatch between input name and UniProt name.

# === 5. SETUP LOGGING ===
# Description: Configures the logging system to record script progress, warnings, and errors.
#              Logs are saved to a file and also printed to the console.

log_file_name = f"annotation_log_{today}.log" # Log file named with the current date.

# Check if the root logger already has handlers (e.g., from previous runs in an interactive environment like Jupyter).
# If handlers exist, reconfigure the logger using force=True. Otherwise, set up basic configuration.
if not logging.getLogger().hasHandlers():
    logging.basicConfig(
        level=logging.INFO, # Set level: INFO, DEBUG, WARNING, ERROR, CRITICAL
        format="%(asctime)s [%(levelname)-8s] %(message)s", # Define log message format.
        datefmt="%Y-%m-%d %H:%M:%S", # Define date format for log entries.
        handlers=[
            logging.FileHandler(log_file_name), # Log to a file.
            logging.StreamHandler()             # Log to the console.
        ]
    )
else:
    # If handlers exist, reconfigure (force=True is important here).
    logging.basicConfig(
        level=logging.INFO, # Ensure level is consistent
        format="%(asctime)s [%(levelname)-8s] %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
        handlers=[
            logging.FileHandler(log_file_name),
            logging.StreamHandler()
        ],
        force=True # Force reconfiguration if logger already exists.
    )

# Log initial script information.
logging.info(f"--- Script Execution Started ---")
logging.info(f"Input file: {INPUT_EXCEL_PATH}")
logging.info(f"Output file: {output_path}")
logging.info(f"Pfam Clan file: {PFAM_CLAN_FILE_PATH}")
logging.info(f"Log file: {log_file_name}")

# === 6. SETUP REQUESTS SESSION (Robust Retry) ===
# Description: Configures a 'requests' session with automatic retries for network robustness.
#              Used for UniProt, STRING, and QuickGO API calls.
#              Helps handle temporary API server issues or rate limits.

session = requests.Session() # Create a session object to persist parameters across requests.
# Define the retry strategy using parameters from CONFIG.
retry_strategy = Retry(
    total=CONFIG["uniprot"]["max_retries"],      # Max number of retries (e.g., 4)
    backoff_factor=CONFIG["uniprot"]["delay"],   # Controls wait time: wait = delay * (2**retry_num) (e.g., 0.5)
    status_forcelist=[429, 500, 502, 503, 504], # Retry on these HTTP status codes.
    allowed_methods=["HEAD", "GET", "OPTIONS"] # Only retry on these safe methods.
)
# Create an HTTP adapter with the retry strategy.
adapter = HTTPAdapter(max_retries=retry_strategy)
# Mount the adapter to handle HTTPS (and optionally HTTP) requests made with this session.
session.mount("https://", adapter)
session.mount("http://", adapter) # Also mount for http if needed.
# Log the configured retry settings.
logging.info(f"Requests session configured with max {CONFIG['uniprot']['max_retries']} retries and backoff factor {CONFIG['uniprot']['delay']}.")

# === 7. LOAD PFAM CLAN TABLE ===
# Description: Loads the Pfam clan information from the specified TSV file.
#              This data is used to map Pfam domain IDs (obtained from UniProt)
#              to their descriptions and clan names. Handles file not found and parsing errors.

pfam_clan_df = None # Initialize variable.
try:
    # Read the potentially gzipped TSV file into a pandas DataFrame.
    pfam_clan_df = pd.read_csv(
        PFAM_CLAN_FILE_PATH,
        sep="\t",               # Tab-separated values.
        header=None,            # No header row in the file.
        names=["Pfam_ID", "Clan_ID", "Clan_Name", "Pfam_Name", "Pfam_Description"], # Assign column names.
        compression='gzip'      # Assumes gzip compression based on common practice/extension.
    )
    logging.info(f"Successfully loaded Pfam clan data ({len(pfam_clan_df)} rows) from {PFAM_CLAN_FILE_PATH}")
except FileNotFoundError:
    # Handle case where the Pfam file is not found; annotations will lack clan info.
    logging.error(f"Pfam clan file not found at: {PFAM_CLAN_FILE_PATH}. PFAM Clan/Description annotations will be incomplete.")
    pfam_clan_df = pd.DataFrame(columns=["Pfam_ID", "Clan_ID", "Clan_Name", "Pfam_Name", "Pfam_Description"]) # Create empty df
except Exception as e:
    # Handle other potential errors during file loading/parsing.
    logging.error(f"Error loading or parsing Pfam clan file {PFAM_CLAN_FILE_PATH}: {e}")
    pfam_clan_df = pd.DataFrame(columns=["Pfam_ID", "Clan_ID", "Clan_Name", "Pfam_Name", "Pfam_Description"]) # Create empty df

# === 8. SETUP GLOBAL CACHES ===
# Description: Initializes dictionaries used as caches to store results from API calls.
#              This avoids redundant requests for the same gene symbol, speeding up the process
#              and reducing load on the external APIs. Caches are cleared at the start if run via `if __name__ == "__main__":`.

global_cache = {
    "uniprot_entry": {},    # Stores the full JSON response from UniProt for each gene symbol.
    "uniprot_basic": {},    # Stores extracted basic fields (ID, desc, function, etc.) from UniProt.
    "uniprot_extra": {},    # Stores extracted extra fields (alt names, keywords, etc.) from UniProt. <<< No UniProt GO >>>
    "gprofiler": {},        # Stores g:Profiler enrichment results for each gene symbol.
    "string_physical": {},  # Stores STRING physical interactors for each gene symbol.
    "string_functional": {},# Stores STRING functional partners for each gene symbol.
    "quickgo_cc": {},       # Stores QuickGO Cellular Component terms for each gene symbol.
    "quickgo_mf": {},       # Stores QuickGO Molecular Function terms for each gene symbol.
    "quickgo_bp": {}        # Stores QuickGO Biological Process terms for each gene symbol.
}
logging.info("Initialized global caches for API results.")


# === QuickGO Fetching Function ===
def fetch_quickgo_annotations(symbol: str) -> None:
    """
    Fetches Gene Ontology (GO) term names (CC, MF, BP) for a given gene symbol
    using the QuickGO REST API, based on its UniProt accession.

    This function first gets the UniProt entry to find the accession, then queries
    QuickGO for annotations associated with that accession, and finally fetches
    the details (name, aspect) for each unique GO ID found. Uses the shared `session`.

    Results are stored directly in the global_cache['quickgo_cc'],
    global_cache['quickgo_mf'], and global_cache['quickgo_bp'] dictionaries,
    keyed by the uppercase gene symbol.

    :param symbol: The gene symbol (e.g., "TP53") to fetch annotations for.
    :type symbol: str
    :return: None. Results are stored in the global cache.
    :rtype: None
    """
    # Validate input symbol
    if not symbol:
        logging.warning("fetch_quickgo_annotations called with empty symbol.")
        return

    # Use uppercase symbol consistent with cache keys
    symbol_upper = symbol.upper()

    # Check cache first (check one, assume all attempted if present)
    if symbol_upper in global_cache["quickgo_cc"]: # Removed check for mf/bp as cc implies attempt
         logging.debug(f"QuickGO annotations already cached for {symbol}.")
         return

    logging.debug(f"Fetching QuickGO annotations for {symbol}...")

    # Get UniProt entry to find the accession number (uses cache if available)
    uniprot_entry = fetch_uniprot_entry(symbol)
    if not uniprot_entry:
        logging.error(f"Cannot fetch QuickGO annotations: no UniProt entry found for {symbol}")
        global_cache["quickgo_cc"][symbol_upper] = pd.NA
        global_cache["quickgo_mf"][symbol_upper] = pd.NA
        global_cache["quickgo_bp"][symbol_upper] = pd.NA
        return

    # Extract primary accession from the UniProt entry
    acc = uniprot_entry.get("primaryAccession")
    if not acc:
        logging.error(f"Cannot fetch QuickGO annotations: no accession found in UniProt entry for {symbol}")
        global_cache["quickgo_cc"][symbol_upper] = pd.NA
        global_cache["quickgo_mf"][symbol_upper] = pd.NA
        global_cache["quickgo_bp"][symbol_upper] = pd.NA
        return

    # Step 1: Get GO IDs associated with the UniProt accession from QuickGO
    annotations_url = "https://www.ebi.ac.uk/QuickGO/services/annotation/search"
    params = {
        "geneProductId": f"UniProtKB:{acc}", # Query by UniProt accession
        "limit": 200                       # Max number of annotations to retrieve
    }
    headers = {"Accept": "application/json"} # Request JSON format

    try:
        # Make the API request using the configured session (includes retry logic)
        r = session.get(annotations_url, headers=headers, params=params, timeout=CONFIG["uniprot"]["timeout"])
        r.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)
        annotations = r.json()
        go_ids = set() # Use a set to store unique GO IDs

        # Extract GO IDs from the results
        for item in annotations.get("results", []):
            go_id = item.get("goId")
            if go_id:
                go_ids.add(go_id)

        if not go_ids:
            logging.warning(f"No QuickGO GO terms found for UniProt accession {acc} ({symbol}).")
            global_cache["quickgo_cc"][symbol_upper] = pd.NA
            global_cache["quickgo_mf"][symbol_upper] = pd.NA
            global_cache["quickgo_bp"][symbol_upper] = pd.NA
            return

        # Step 2: For each unique GO ID, fetch its name and aspect (BP, MF, CC)
        bp_terms = set()
        mf_terms = set()
        cc_terms = set()

        logging.debug(f"Found {len(go_ids)} unique GO IDs for {symbol}. Fetching term details...")
        # Loop through each unique GO ID found
        for go_id in go_ids:
            term_url = f"https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/{go_id}"
            # Request term details using the configured session
            term_resp = session.get(term_url, headers=headers, timeout=CONFIG["uniprot"]["timeout"])

            if not term_resp.ok:
                logging.warning(f"Failed to fetch QuickGO term details for {go_id} (Symbol: {symbol}, Status: {term_resp.status_code})")
                continue # Skip this term if fetching fails

            term_data = term_resp.json()
            results = term_data.get("results", [])
            if not results:
                logging.warning(f"No results found in QuickGO term details response for {go_id} (Symbol: {symbol})")
                continue # Skip if no results structure

            # Extract name and aspect from the first result entry
            term_info = results[0]
            name = term_info.get("name")
            aspect = term_info.get("aspect") # 'biological_process', 'molecular_function', or 'cellular_component'

            if not name or not aspect:
                logging.warning(f"Missing name or aspect for GO term {go_id} (Symbol: {symbol})")
                continue # Skip if essential info is missing

            # Add the term name to the appropriate set based on its aspect
            if aspect == "biological_process":
                bp_terms.add(name)
            elif aspect == "molecular_function":
                mf_terms.add(name)
            elif aspect == "cellular_component":
                cc_terms.add(name)

            # Optional: Small delay can be added here if hitting rate limits *within* this loop specifically
            # time.sleep(0.02) # Example: 20ms delay

        # Store the collected terms in the global cache, joined by semicolons
        global_cache["quickgo_cc"][symbol_upper] = "; ".join(sorted(cc_terms)) if cc_terms else pd.NA
        global_cache["quickgo_mf"][symbol_upper] = "; ".join(sorted(mf_terms)) if mf_terms else pd.NA
        global_cache["quickgo_bp"][symbol_upper] = "; ".join(sorted(bp_terms)) if bp_terms else pd.NA
        logging.debug(f"Successfully cached QuickGO annotations for {symbol}.")

    except requests.exceptions.RequestException as e:
        # Handle errors during the API requests
        logging.error(f"QuickGO API request failed for {symbol} (Accession: {acc}): {e}")
        global_cache["quickgo_cc"][symbol_upper] = pd.NA
        global_cache["quickgo_mf"][symbol_upper] = pd.NA
        global_cache["quickgo_bp"][symbol_upper] = pd.NA
    except Exception as e:
        # Handle unexpected errors during processing
        logging.error(f"Unexpected error during QuickGO annotation fetching for {symbol} (Accession: {acc}): {e}")
        global_cache["quickgo_cc"][symbol_upper] = pd.NA
        global_cache["quickgo_mf"][symbol_upper] = pd.NA
        global_cache["quickgo_bp"][symbol_upper] = pd.NA


# === 9. UTILITY FUNCTIONS ===
# Description: Helper functions for common data manipulation tasks like extracting gene symbols
#              and processing Pfam domain information.

def extract_main_gene_symbol(name: typing.Optional[str]) -> typing.Optional[str]:
    """
    Extracts the primary gene symbol from a string, typically the first element
    if the string is semicolon or space-separated. Returns the symbol in uppercase.
    Handles potential NaN, None, or empty string inputs.

    :param name: The input string, potentially containing multiple gene names.
    :type name: typing.Optional[str]
    :return: The extracted primary gene symbol in uppercase, or None if input is invalid/empty.
    :rtype: typing.Optional[str]
    """
    if pd.isna(name) or not isinstance(name, str) or not name.strip():
        return None # Return None for NaN, non-string, or empty/whitespace-only strings.
    # Replace semicolons with spaces, split by whitespace, take the first part, strip whitespace, and convert to uppercase.
    return name.replace(';', ' ').split()[0].strip().upper()

def extract_pfam_domains_from_uniprot_entry(entry: dict) -> str:
    """
    Extracts Pfam domain IDs from the 'uniProtKBCrossReferences' section
    of a UniProt API JSON entry.

    :param entry: The UniProt entry JSON object (as a dictionary).
    :type entry: dict
    :return: A semicolon-separated string of unique, sorted Pfam IDs, or an empty string if none found.
    :rtype: str
    """
    pfams = set() # Use a set to automatically handle duplicates.
    if not entry or not isinstance(entry, dict):
        return "" # Return empty if the entry is missing or not a dictionary.

    # Iterate through cross-references in the UniProt entry.
    for ref in entry.get("uniProtKBCrossReferences", []):
        # Check if the reference is a dictionary, is from the 'Pfam' database, and has an 'id'.
        if isinstance(ref, dict) and ref.get("database") == "Pfam" and ref.get("id"):
            pfams.add(ref.get("id")) # Add the Pfam ID to the set.

    # Return the sorted Pfam IDs joined by semicolons.
    return "; ".join(sorted(list(pfams))) if pfams else ""

def get_pfam_metadata(pfam_ids_string: typing.Optional[str]) -> typing.Tuple[str, str]:
    """
    Looks up Pfam descriptions and clan names for a given string of semicolon-separated Pfam IDs,
    using the pre-loaded Pfam clan DataFrame (`pfam_clan_df`).

    :param pfam_ids_string: A string containing one or more Pfam IDs separated by semicolons.
    :type pfam_ids_string: typing.Optional[str]
    :return: A tuple containing two strings:
             1. Semicolon-separated unique Pfam descriptions.
             2. Semicolon-separated unique Pfam clan names.
             Returns ("", "") if input is invalid, no IDs match, or `pfam_clan_df` is unavailable.
    :rtype: typing.Tuple[str, str]
    """
    # Check if input string is valid and the Pfam DataFrame is loaded and not empty.
    if not pfam_ids_string or pfam_clan_df is None or pfam_clan_df.empty:
        return "", ""

    try:
        # Split the input string into a set of unique, stripped Pfam IDs.
        pfam_ids = {pfam.strip() for pfam in pfam_ids_string.split(";") if pfam.strip()}
        if not pfam_ids: return "", "" # Return empty if no valid IDs after splitting/stripping.

        # Check if the required column exists in the DataFrame.
        if "Pfam_ID" not in pfam_clan_df.columns:
             logging.warning("Pfam_ID column not found in Pfam clan data. Cannot fetch metadata.")
             return "", ""

        # Filter the Pfam DataFrame to find rows matching the extracted Pfam IDs.
        matched = pfam_clan_df[pfam_clan_df["Pfam_ID"].isin(pfam_ids)].copy()

        # Extract unique, non-null descriptions and sort them.
        descriptions = "; ".join(sorted(matched["Pfam_Description"].dropna().astype(str).unique()))
        # Extract unique, non-null clan names and sort them.
        clans = "; ".join(sorted(matched["Clan_Name"].dropna().astype(str).unique()))

        return descriptions, clans
    except Exception as e:
        # Log any error during the lookup process.
        logging.error(f"Error getting Pfam metadata for IDs '{pfam_ids_string}': {e}")
        return "", ""

# === 10. FETCH and PROCESS FUNCTIONS ===
# Description: Functions dedicated to fetching data from external APIs (UniProt, STRING, g:Profiler)
#              and processing the raw responses into structured information suitable for the output table.
#              These functions utilize the global caches to store and retrieve results.

# --- UniProt Data Fetching and Processing ---

def fetch_uniprot_entry(symbol: typing.Optional[str]) -> typing.Optional[dict]:
    """
    Fetches the full UniProt JSON entry for a given gene symbol from the UniProtKB database.
    It specifically searches for reviewed (Swiss-Prot) entries for Homo sapiens (organism ID 9606).
    Handles caching and potential multiple matches by preferring non-isoform entries.
    Uses the shared `session` with retry logic.

    :param symbol: The gene symbol (e.g., "TP53").
    :type symbol: typing.Optional[str]
    :return: The UniProt entry as a dictionary if found, otherwise None. Results are cached.
    :rtype: typing.Optional[dict]
    """
    if not symbol: return None # Return None if no symbol is provided.
    symbol_upper = symbol.upper() # Use uppercase for consistent cache keys.

    # Check cache first.
    if symbol_upper in global_cache["uniprot_entry"]:
        return global_cache["uniprot_entry"][symbol_upper]

    logging.debug(f"Fetching UniProt entry for {symbol}...")
    try:
        # Construct the UniProt search query.
        query = f"gene_exact:{symbol_upper} AND organism_id:9606 AND reviewed:true"
        search_params = {
            "query": query,
            "format": "json",
            "fields": "accession", # Only request the accession field initially for search.
            "size": 2              # Limit results (helps identify multiple hits quickly).
        }
        search_url = "https://rest.uniprot.org/uniprotkb/search"

        # Perform the search request using the configured session.
        search_resp = session.get(search_url, params=search_params, timeout=CONFIG["uniprot"]["timeout"])
        search_resp.raise_for_status() # Check for HTTP errors.
        results = search_resp.json().get("results", [])

        if not results:
            # Log if no reviewed entry is found for the symbol.
            logging.warning(f"No reviewed UniProt entry found for gene symbol: {symbol}")
            global_cache["uniprot_entry"][symbol_upper] = None; return None

        acc = None # Variable to store the selected UniProt accession.
        # Handle multiple search results.
        if len(results) > 1:
            # Prefer entries whose accession does not contain '-' (likely canonical forms).
            non_isoform = [r for r in results if isinstance(r.get("primaryAccession"), str) and '-' not in r.get("primaryAccession")]
            acc = non_isoform[0]["primaryAccession"] if non_isoform else results[0]["primaryAccession"]
            logging.info(f"Multiple UniProtKB hits for {symbol}. Using entry: {acc}")
        else:
            # If only one result, use its accession.
            acc = results[0]["primaryAccession"]

        # Fetch the full entry using the selected accession.
        entry_url = f"https://rest.uniprot.org/uniprotkb/{acc}.json"
        entry_resp = session.get(entry_url, timeout=CONFIG["uniprot"]["timeout"])
        entry_resp.raise_for_status() # Check for HTTP errors on the entry request.
        entry_data = entry_resp.json()

        # Cache the fetched entry data.
        global_cache["uniprot_entry"][symbol_upper] = entry_data
        logging.debug(f"Successfully fetched/cached UniProt entry for {symbol} (Acc: {acc})")
        return entry_data

    except requests.exceptions.RequestException as e:
        # Handle specific 404 errors vs general request errors.
        if isinstance(e, requests.exceptions.HTTPError) and e.response.status_code == 404 and 'entry_url' in locals():
             logging.warning(f"UniProt entry request for {symbol} (Accession: {acc}) resulted in 404 Not Found. URL: {entry_url}")
        else:
             logging.error(f"UniProt request failed for {symbol}: {e}")
        global_cache["uniprot_entry"][symbol_upper] = None; return None
    except Exception as e:
        # Handle any other unexpected errors.
        logging.error(f"Unexpected error fetching UniProt entry for {symbol}: {e}")
        global_cache["uniprot_entry"][symbol_upper] = None; return None

def process_uniprot_basic_fields(symbol: str, entry: dict) -> dict:
    """
    Processes a fetched UniProt JSON entry to extract a predefined set of basic annotation fields
    (ID, Description, Function, Location, Pfam info, Mass, Canonical status).
    Handles data extraction from nested JSON structures and looks up Pfam metadata.

    :param symbol: The gene symbol (used for logging and cache keys).
    :type symbol: str
    :param entry: The UniProt entry dictionary obtained from `fetch_uniprot_entry`.
    :type entry: dict
    :return: A dictionary where keys are the basic column name constants (e.g., COL_UNIPROT_ID)
             and values are the extracted data (or pd.NA if not found/applicable). Results are cached.
    :rtype: dict
    """
    symbol_upper = symbol.upper() # Use uppercase for cache keys.

    # Check cache first.
    if symbol_upper in global_cache["uniprot_basic"]:
        return global_cache["uniprot_basic"][symbol_upper]

    # Define the list of basic columns to populate.
    basic_cols_list = [
        COL_UNIPROT_ID, COL_PROTEIN_DESC, COL_FUNCTION, COL_LOCATION,
        COL_PFAM_DOMAINS, COL_PFAM_DESC, COL_PFAM_CLANS, COL_CANONICAL, COL_MASS_KDA
    ]
    # Initialize the result dictionary with NA values.
    result = {col: pd.NA for col in basic_cols_list}

    # Return default NA values if the entry is missing or invalid.
    if not entry or not isinstance(entry, dict):
        logging.debug(f"Cannot process basic fields for {symbol}, entry missing or invalid.")
        global_cache["uniprot_basic"][symbol_upper] = result; return result

    logging.debug(f"Processing basic UniProt fields for {symbol}...")
    try:
        # --- Extract Basic Fields ---
        # UniProt ID (Primary Accession)
        acc = entry.get("primaryAccession", pd.NA)
        result[COL_UNIPROT_ID] = acc

        # Protein Description (Recommended Name, fallback to Submission Name)
        name = entry.get("proteinDescription", {}).get("recommendedName", {}).get("fullName", {}).get("value", "")
        if not name: # If recommendedName is not present, try the first submissionName
            submission_names = entry.get("proteinDescription", {}).get("submissionNames", [])
            if submission_names and isinstance(submission_names[0], dict):
                 name = submission_names[0].get("fullName", {}).get("value", "")
        result[COL_PROTEIN_DESC] = name if name else pd.NA

        # Function (from FUNCTION comments)
        function_text = pd.NA
        for comment in entry.get("comments", []):
             if isinstance(comment, dict) and comment.get("commentType") == "FUNCTION":
                texts = comment.get("texts")
                # Extract the 'value' from the first text entry if available.
                if isinstance(texts, list) and len(texts) > 0:
                    first_text = texts[0]
                    if isinstance(first_text, dict):
                        function_text = first_text.get("value", pd.NA)
                        break # Found the function comment, no need to check others.
        result[COL_FUNCTION] = function_text

        # Molecular Mass (converted to kDa)
        mass = entry.get("sequence", {}).get("molWeight", None)
        mass_kda = pd.NA
        if mass is not None:
            try:
                mass_kda = round(float(mass) / 1000, 2) # Calculate kDa and round.
            except (ValueError, TypeError):
                logging.warning(f"Could not convert mass '{mass}' to float for {symbol}. Setting to NA.")
        result[COL_MASS_KDA] = mass_kda

        # Subcellular Location (from SUBCELLULAR LOCATION comments)
        locations = set() # Use set for unique locations.
        for comment in entry.get("comments", []):
             if isinstance(comment, dict) and comment.get("commentType") == "SUBCELLULAR LOCATION":
                 locs_data = comment.get("subcellularLocations", [])
                 if isinstance(locs_data, list):
                     for loc_entry in locs_data:
                          if isinstance(loc_entry, dict):
                              location_info = loc_entry.get("location")
                              if isinstance(location_info, dict):
                                  value = location_info.get("value")
                                  if value: locations.add(value) # Add valid location value.
        result[COL_LOCATION] = "; ".join(sorted(list(locations))) if locations else pd.NA

        # Pfam Domains (extracted using helper function)
        pfam_str = extract_pfam_domains_from_uniprot_entry(entry)
        result[COL_PFAM_DOMAINS] = pfam_str if pfam_str else pd.NA

        # Pfam Descriptions and Clans (looked up using helper function)
        pfam_desc, clan_name = get_pfam_metadata(pfam_str)
        result[COL_PFAM_DESC] = pfam_desc if pfam_desc else pd.NA
        result[COL_PFAM_CLANS] = clan_name if clan_name else pd.NA

        # Canonical Isoform Check (heuristic based on accession format)
        # Assumes accessions with '-' are isoforms, unless it's '-1' (often the reference).
        is_canonical = "No" if isinstance(acc, str) and '-' in acc and '-1' not in acc else "Yes"
        result[COL_CANONICAL] = is_canonical

        # Cache the processed basic fields.
        global_cache["uniprot_basic"][symbol_upper] = result; return result

    except Exception as e:
        # Log errors during processing and return default NA values.
        logging.error(f"Error processing basic UniProt fields for {symbol}: {e}")
        result_error = {col: pd.NA for col in basic_cols_list}
        global_cache["uniprot_basic"][symbol_upper] = result_error; return result_error

def process_uniprot_extra_fields(symbol: str, entry: dict) -> dict:
    """
    Processes a fetched UniProt JSON entry to extract additional annotation fields:
    Alternative names, catalytic activity, diseases, similarities, pathways,
    protein existence level, and categorized keywords.
    <<< MODIFIED: No longer processes UniProt GO terms. >>>

    Handles data extraction and formats results appropriately (e.g., semicolon-separated strings).
    Dynamically creates column keys for categorized keywords (e.g., "UniProt KW: Category").

    :param symbol: The gene symbol (used for logging and cache keys).
    :type symbol: str
    :param entry: The UniProt entry dictionary obtained from `fetch_uniprot_entry`.
    :type entry: dict
    :return: A dictionary where keys are the extra column name constants (e.g., COL_ALT_NAMES_V2)
             or dynamically generated keyword column names, and values are the extracted data
             (or pd.NA if not found/applicable). Results are cached.
    :rtype: dict
    """
    symbol_upper = symbol.upper() # Use uppercase for cache keys.

    # Check cache first.
    if symbol_upper in global_cache["uniprot_extra"]:
        return global_cache["uniprot_extra"][symbol_upper]

    # Define the *static* extra columns expected
    # <<< MODIFIED: Removed UniProt GO columns >>>
    static_extra_cols_keys = [
        COL_ALT_NAMES_V2, COL_CAT_ACT_V2, COL_DISEASES_V2,
        COL_SIMILARITIES_V2, COL_PATHWAYS_V2, COL_EXISTENCE_V2,
    ]
    # Initialize a dictionary to return in case of errors.
    result_on_error = {col: pd.NA for col in static_extra_cols_keys}

    # Return default NA values if the entry is missing or invalid.
    if not entry or not isinstance(entry, dict):
        logging.debug(f"Cannot process extra fields for {symbol}, entry missing or invalid.")
        global_cache["uniprot_extra"][symbol_upper] = result_on_error
        return result_on_error

    logging.debug(f"Processing extra UniProt fields for {symbol}...")
    try:
        results = {} # Initialize dictionary to store extracted extra fields.

        # --- Alternative Names ---
        alt_names = set()
        protein_desc = entry.get("proteinDescription", {})
        rec_name_alts = protein_desc.get("alternativeNames", [])
        for alt in rec_name_alts:
            if isinstance(alt, dict): val = alt.get("fullName", {}).get("value");
            if val: alt_names.add(val)
        sub_names = protein_desc.get("submissionNames", [])
        if len(sub_names) > 1:
             for sub in sub_names[1:]:
                 if isinstance(sub, dict): val = sub.get("fullName", {}).get("value");
                 if val: alt_names.add(val)
        results[COL_ALT_NAMES_V2] = "; ".join(sorted(list(alt_names))) if alt_names else pd.NA

        # --- Protein Existence Level ---
        existence_mapping = {
            "ECO:0000269": "1: Evidence at protein level", "ECO:0000314": "1: Evidence at protein level",
            "ECO:0000250": "1: Evidence at protein level", "ECO:0000303": "1: Evidence at protein level",
            "ECO:0000305": "2: Evidence at transcript level", "ECO:0000255": "2: Evidence at transcript level",
            "ECO:0000315": "2: Evidence at transcript level", "ECO:0000256": "3: Inferred from homology",
            "ECO:0000313": "3: Inferred from homology", "ECO:0000307": "4: Predicted",
            "ECO:0000213": "5: Uncertain"
        }
        protein_existence_info = entry.get("proteinExistence")
        existence_str = pd.NA
        if isinstance(protein_existence_info, dict):
            protein_existence_code = protein_existence_info.get("evidenceCode")
            if protein_existence_code: existence_str = existence_mapping.get(protein_existence_code, protein_existence_code)
        elif isinstance(protein_existence_info, str): existence_str = existence_mapping.get(protein_existence_info, protein_existence_info)
        elif protein_existence_info is not None: logging.warning(f"Unexpected type ({type(protein_existence_info).__name__}) for 'proteinExistence' in {symbol}. Value: '{protein_existence_info}'. Setting to NA.")
        results[COL_EXISTENCE_V2] = existence_str

        # --- Process Comments (Catalytic Activity, Diseases, Similarities, Pathways) ---
        catalytic_activity, diseases, similarities, pathways = set(), set(), set(), set()
        for comment in entry.get("comments", []):
            if not isinstance(comment, dict): continue
            ctype = comment.get("commentType", "")
            try:
                if ctype == "CATALYTIC ACTIVITY":
                    reaction = comment.get("reaction", {})
                    if isinstance(reaction, dict): name = reaction.get("name");
                    if name: catalytic_activity.add(name)
                elif ctype == "DISEASE":
                    disease_entry = comment.get("disease", {})
                    if isinstance(disease_entry, dict):
                        disease_id, acronym, desc = disease_entry.get("diseaseId"), disease_entry.get("acronym"), disease_entry.get("description")
                        if disease_id and acronym: diseases.add(f"{disease_id} ({acronym}): {desc}" if desc else f"{disease_id} ({acronym})")
                        elif desc: diseases.add(desc)
                    elif comment.get("texts"):
                          for item in comment.get("texts", []): val = item.get("value") if isinstance(item, dict) else item if isinstance(item, str) else None;
                          if val: diseases.add(str(val).strip())
                elif ctype == "SIMILARITY":
                     texts = comment.get("texts")
                     if isinstance(texts, list):
                         for item in texts: val = item.get("value") if isinstance(item, dict) else item if isinstance(item, str) else None;
                         if val: similarities.add(str(val).strip())
                elif ctype == "PATHWAY":
                     texts = comment.get("texts")
                     if isinstance(texts, list):
                         for item in texts: val = item.get("value") if isinstance(item, dict) else item if isinstance(item, str) else None;
                         if val: pathways.add(str(val).strip())
            except Exception as com_e:
                logging.warning(f"Error processing comment type '{ctype}' for {symbol}: {com_e}")
        results[COL_CAT_ACT_V2] = "; ".join(sorted(list(catalytic_activity))) if catalytic_activity else pd.NA
        results[COL_DISEASES_V2] = "; ".join(sorted(list(diseases))) if diseases else pd.NA
        results[COL_SIMILARITIES_V2] = "; ".join(sorted(list(similarities))) if similarities else pd.NA
        results[COL_PATHWAYS_V2] = "; ".join(sorted(list(pathways))) if pathways else pd.NA

        # --- Process Categorized Keywords ---
        categorized_keywords = {} # Dictionary to hold {Column Name: Keywords String}
        raw_keywords = entry.get("keywords", [])
        if isinstance(raw_keywords, list):
            temp_kw_dict = {} # Temporary dict {Category: set(Keywords)}
            for kw_item in raw_keywords:
                if isinstance(kw_item, dict):
                    category = kw_item.get("category")
                    name = kw_item.get("name") or kw_item.get("id") # Use name, fallback to ID.
                    if category and name:
                        clean_category = category.replace(' ', '_') # Make category suitable for column name.
                        if clean_category not in temp_kw_dict: temp_kw_dict[clean_category] = set()
                        temp_kw_dict[clean_category].add(name)
                    elif name: # Add keywords without a category to "Uncategorized".
                        if "Uncategorized" not in temp_kw_dict: temp_kw_dict["Uncategorized"] = set()
                        temp_kw_dict["Uncategorized"].add(name)
            # Format the keyword columns for the final results dictionary.
            for category, kw_set in temp_kw_dict.items():
                col_name = f"UniProt KW: {category}" # Create dynamic column name.
                categorized_keywords[col_name] = "; ".join(sorted(list(kw_set)))
        # Add the generated keyword columns to the main results dictionary.
        results.update(categorized_keywords)

        # <<< REMOVED UniProt GO Term Processing Section >>>
        # The loop iterating through `uniProtKBCrossReferences` for GO terms is no longer needed here.

        # --- Final Cache and Return ---
        # Ensure all *statically* defined extra columns are present in the results, even if empty.
        for key in static_extra_cols_keys:
            if key not in results:
                results[key] = pd.NA

        # Cache the processed extra fields.
        global_cache["uniprot_extra"][symbol_upper] = results
        return results

    except Exception as e:
        # Log errors during processing. Use `logging.exception` to include traceback.
        logging.exception(f"Error processing extra UniProt fields for {symbol}: {e}")
        # Cache the error state (NA values for static columns).
        global_cache["uniprot_extra"][symbol_upper] = result_on_error
        return result_on_error


# --- STRING DB Data Fetching ---

def fetch_string_interactors(symbol: typing.Optional[str], network_type: str = "physical") -> str:
    """
    Fetches high-confidence (score >= 700) interacting proteins for a given gene symbol
    from the STRING database API (v11 or later). Uses the shared `session`.

    :param symbol: The gene symbol (e.g., "TP53").
    :type symbol: typing.Optional[str]
    :param network_type: Type of interaction network ('physical' or 'functional'). Defaults to 'physical'.
    :type network_type: str
    :return: A semicolon-separated string of unique, sorted interactor gene symbols, or an empty string
             if no interactors are found or an error occurs. Results are cached.
    :rtype: str
    """
    if not symbol: return "" # Return empty if no symbol provided.
    symbol_upper = symbol.upper() # Use uppercase for consistency.
    cache_key = f"string_{network_type}" # Dynamic cache key based on network type.

    # Check cache first.
    if symbol_upper in global_cache[cache_key]:
        return global_cache[cache_key][symbol_upper]

    logging.debug(f"Fetching STRING ({network_type}) interactors for {symbol}...")
    result_on_error = "" # Default return value on error.

    try:
        # STRING API endpoint for network interactions.
        url = "https://string-db.org/api/tsv/network"
        # Parameters for the API request.
        params = {
            "identifiers": symbol,         # The query gene symbol.
            "species": 9606,               # Homo sapiens NCBI taxonomy ID.
            "required_score": 700,         # Minimum interaction score (0-1000, 700=high confidence).
            "network_type": network_type,  # 'physical' or 'functional'.
            "limit": 200,                  # Maximum number of interactors to return.
            "caller_identity": "PythonProteomicsScript_UL82" # Identify the script to STRING DB admins.
        }

        # Make the request using the configured session.
        resp = session.get(url, params=params, timeout=CONFIG["uniprot"]["timeout"])
        resp.raise_for_status() # Check for HTTP errors.

        interactors = set() # Use a set for unique interactors.
        lines = resp.text.strip().split("\n") # Split TSV response into lines.

        # Process lines (skip header line[0]).
        if len(lines) > 1:
            query_symbol_upper_check = symbol.upper() # Ensure case-insensitive comparison.
            for line in lines[1:]:
                parts = line.split("\t")
                # Ensure the line has enough columns (at least 4 for protein1, protein2).
                if len(parts) >= 4:
                    # Preferred names are usually in columns 2 and 3 (0-indexed).
                    name1_api, name2_api = parts[2], parts[3] # Get potential interactor names
                    name1_upper, name2_upper = name1_api.upper(), name2_api.upper() # Uppercase for comparison

                    # Add the interactor that is *not* the query symbol itself.
                    if name1_upper == query_symbol_upper_check and name2_upper != query_symbol_upper_check:
                        interactors.add(name2_api) # Add the original case name
                    elif name2_upper == query_symbol_upper_check and name1_upper != query_symbol_upper_check:
                        interactors.add(name1_api) # Add the original case name

        # Format the result string.
        result = "; ".join(sorted(list(interactors)))

        # Cache the result.
        global_cache[cache_key][symbol_upper] = result
        logging.debug(f"Successfully fetched {len(interactors)} STRING ({network_type}) interactors for {symbol}.")
        return result

    except requests.exceptions.RequestException as e:
        # Log request errors.
        logging.error(f"STRING ({network_type}) request failed for {symbol}: {e}")
        global_cache[cache_key][symbol_upper] = result_on_error; return result_on_error
    except Exception as e:
        # Log other unexpected errors during processing.
        logging.error(f"Unexpected error processing STRING ({network_type}) data for {symbol}: {e}")
        global_cache[cache_key][symbol_upper] = result_on_error; return result_on_error


# --- g:Profiler Enrichment Analysis ---

# Initialize the g:Profiler client.
try:
    # Update user agent to be more specific
    gp = GProfiler(return_dataframe=True, user_agent="PythonProteomicsScript_UL82/1.2")
    logging.info("g:Profiler client initialized.")
    gprofiler_available = True # Flag indicating g:Profiler is ready.
except Exception as e:
    logging.error(f"Failed to initialize g:Profiler client: {e}. g:Profiler annotations will be skipped.")
    gp = None; gprofiler_available = False # Set flag to False if initialization fails.

def fetch_gprofiler_enrichment(symbol: typing.Optional[str]) -> dict:
    """
    Performs functional enrichment analysis for a *single* gene symbol using the g:Profiler API.
    Queries the sources defined in CONFIG['gprofiler']['sources'] and filters by the p-value threshold.
    Does NOT use the shared session's retry logic; relies on the gprofiler library's handling.

    Note: g:Profiler is typically used for gene *lists*, but here it's applied per gene
          to get associated terms from various databases (GO, KEGG, REAC, etc.) without
          calculating enrichment significance in the traditional sense.

    :param symbol: The gene symbol (e.g., "TP53").
    :type symbol: typing.Optional[str]
    :return: A dictionary where keys are formatted source names (e.g., "GO:BP Terms") and
             values are semicolon-separated strings of the top associated term names (up to max_terms),
             or pd.NA if no terms are found for a source or if g:Profiler is unavailable/fails.
             Results are cached.
    :rtype: dict
    """
    # Return empty if no symbol or g:Profiler client isn't available.
    if not symbol or not gprofiler_available: return {}
    symbol_upper = symbol.upper() # Use uppercase for cache keys.

    # Check cache first.
    if symbol_upper in global_cache["gprofiler"]:
        return global_cache["gprofiler"][symbol_upper]

    logging.debug(f"Fetching g:Profiler enrichment for {symbol}...")
    gprofiler_sources = CONFIG["gprofiler"]["sources"]
    # Define the structure for failure cases (all sources NA).
    enrichment_failure = {f"{s} Terms": pd.NA for s in gprofiler_sources}

    try:
        # Perform the profile request using the g:Profiler client.
        results_df = gp.profile(
            organism="hsapiens",                 # Human organism.
            query=[symbol],                      # The single gene symbol as a list.
            no_evidences=False,                  # Include evidence codes (though not explicitly used here).
            user_threshold=CONFIG["gprofiler"]["threshold"], # Significance threshold.
            sources=gprofiler_sources,           # Databases to query.
            all_results=False                    # Get only significant results based on threshold.
        )

        # Process the resulting DataFrame.
        if results_df is None or results_df.empty:
            # If no significant terms are found, use the failure structure.
            enrichment = enrichment_failure
        else:
            enrichment = {}
            # Iterate through each requested source (e.g., GO:BP, REAC).
            for source in gprofiler_sources:
                # Filter results for the current source and sort by p-value.
                source_results = results_df[results_df["source"] == source].sort_values(by="p_value")
                # Get the top 'max_terms' unique term names.
                terms = source_results["name"].dropna().unique().tolist()[:CONFIG["gprofiler"]["max_terms"]]
                # Format the column name and join terms with semicolons.
                enrichment[f"{source} Terms"] = "; ".join(terms) if terms else pd.NA
            # Ensure all requested source columns exist in the final dict, even if empty.
            for source in gprofiler_sources:
                if f"{source} Terms" not in enrichment:
                    enrichment[f"{source} Terms"] = pd.NA

        # Cache the results.
        global_cache["gprofiler"][symbol_upper] = enrichment
        logging.debug(f"Successfully fetched/cached g:Profiler data for {symbol}.")
        return enrichment

    except Exception as e:
        # Log errors during the g:Profiler request or processing.
        logging.error(f"g:Profiler fetch/processing failed for {symbol}: {e}")
        # Cache the failure state.
        global_cache["gprofiler"][symbol_upper] = enrichment_failure; return enrichment_failure


# === 11. BACKGROUND DETECTION FUNCTIONS ===
# Description: Functions to apply heuristic flags for identifying potential background proteins
#              based on keywords or expression patterns in control experiments (EVs).

def detect_sticky_binder(protein_desc: typing.Optional[str]) -> str:
    """
    Checks if a protein description contains any keywords from the predefined
    `CONFIG["background_keywords"]` list (case-insensitive).

    :param protein_desc: The protein description string (usually from UniProt).
    :type protein_desc: typing.Optional[str]
    :return: "Sticky Background" if a keyword is found, otherwise an empty string.
    :rtype: str
    """
    # Return empty if description is missing, NaN, or not a string.
    if pd.isna(protein_desc) or not isinstance(protein_desc, str): return ""

    desc_lower = protein_desc.lower() # Convert to lowercase for case-insensitive matching.
    # Check for each keyword in the description.
    for keyword in CONFIG["background_keywords"]:
        if keyword.lower() in desc_lower:
            return "Sticky Background" # Return flag immediately if found.
    return "" # Return empty if no keywords match.

def detect_ev_lfq_background(row: pd.Series) -> str:
    """
    Flags proteins as potential Extracellular Vesicle (EV) background based on
    LFQ intensities in EV control columns and the log2 Fold Change (log2FC).

    The logic is: If the median intensity across specified EV columns (`CONFIG["ev_columns"]`)
    is > 0 AND the log2FC is < 1.0 (or missing/NaN), flag as "Likely EV Background".

    Requires `CONFIG["ev_columns"]` and `COL_LOG2FC` to be present in the input Series (row).

    :param row: A pandas Series representing a row from the DataFrame.
    :type row: pd.Series
    :return: "Likely EV Background" if conditions are met, otherwise an empty string.
    :rtype: str
    """
    ev_cols = CONFIG["ev_columns"] # Get EV column names from config.
    # Check if all required EV columns and the log2FC column exist in the row's index.
    if not set(ev_cols).issubset(row.index):
        return "" # Silently return empty if columns are missing
    if COL_LOG2FC not in row.index:
        return "" # Silently return empty if columns are missing

    try:
        # Calculate the median of the EV columns, coercing errors to NaN.
        median_ev = pd.to_numeric(row[ev_cols], errors='coerce').median()
        # Get the log2FC value, coercing errors to NaN.
        log2fc = pd.to_numeric(row.get(COL_LOG2FC), errors='coerce')

        # Apply the background logic.
        if pd.notna(median_ev) and median_ev > 0 and (pd.isna(log2fc) or log2fc < 1.0):
            return "Likely EV Background"
        return "" # Return empty if conditions are not met.
    except Exception as e:
        # Log errors during the calculation.
        logging.error(f"Error during EV LFQ background detection for row (Index: {row.name}): {e}")
        return "" # Return empty on error.


# === 12. IMMEDIATE OUTPUT SAFETY FUNCTION ===
# Description: A function designed to write DataFrames to Excel sheets robustly,
#              handling potential issues like invalid characters in sheet names,
#              data type problems that prevent writing, and ensuring the file is saved
#              after each sheet write (for partial results in case of later errors).

def write_sheet_safely(writer: pd.ExcelWriter, sheet_name: str, df: pd.DataFrame):
    """
    Writes a DataFrame to a specified sheet in an Excel file using an existing ExcelWriter.

    Includes safety checks:
    - Cleans the sheet name (removes invalid chars, truncates to 31 chars).
    - Attempts to convert object columns to strings before writing.
    - Saves the workbook immediately after writing the sheet.
    - Logs progress and errors.

    :param writer: The pandas ExcelWriter object.
    :type writer: pd.ExcelWriter
    :param sheet_name: The desired name for the sheet.
    :type sheet_name: str
    :param df: The pandas DataFrame to write.
    :type df: pd.DataFrame
    :return: None
    :rtype: None
    """
    # Define regex for characters invalid in Excel sheet names.
    invalid_excel_chars = r'[\\*?:/\[\]]'
    # Remove invalid characters and truncate sheet name to Excel's limit (31 chars).
    safe_sheet_name = re.sub(invalid_excel_chars, '_', sheet_name)[:31]

    try:
        logging.info(f"Writing sheet '{safe_sheet_name}' ({df.shape[0]} rows, {df.shape[1]} cols).")
        # Work on a copy to avoid modifying the original DataFrame.
        df_copy = df.copy()

        # Attempt to convert object columns to strings to prevent potential write errors.
        for col in df_copy.select_dtypes(include=['object']).columns:
            try:
                # Fill NaNs with empty string and convert column to string type.
                df_copy[col] = df_copy[col].fillna('').astype(str)
            except Exception as e:
                # If bulk conversion fails, try element-wise conversion as a fallback.
                logging.warning(f"Could not bulk convert column '{col}' to string in sheet '{safe_sheet_name}': {e}. Trying element-wise.")
                df_copy[col] = df_copy[col].apply(lambda x: str(x) if pd.notna(x) else '')

        # Crucial check: Ensure the writer's underlying workbook object exists.
        if not hasattr(writer, 'book') or writer.book is None:
            logging.error(f"ExcelWriter's 'book' attribute is missing or None. Cannot write sheet '{safe_sheet_name}'.")
            return # Cannot proceed without a valid workbook.

        # Write the DataFrame to the Excel sheet.
        df_copy.to_excel(
            writer,
            sheet_name=safe_sheet_name,
            index=False,        # Do not write DataFrame index as a column.
            freeze_panes=(1, 0) # Freeze the header row.
        )

        # Save the workbook immediately after writing the sheet.
        # Ensures partial results if script fails later. Can be slow.
        writer.book.save(output_path)
        logging.info(f"Successfully wrote and saved sheet '{safe_sheet_name}' to {output_path}.")

    except Exception as e:
        # Log any exceptions during writing or saving.
        logging.exception(f"Failed to write or save sheet '{safe_sheet_name}': {e}")


# === 13. MAIN PROCESSING FUNCTION ===
# Description: Orchestrates the entire annotation process:
#              - Loads the input Excel file.
#              - Sets up the output Excel writer.
#              - Pre-loads background gene lists if configured.
#              - Iterates through each sheet in the input file.
#              - For each sheet:
#                  - Checks for presence of 'Significant' column.
#                  - If 'Significant' is present, filters significant proteins/genes (+).
#                  - If 'Significant' is absent, processes all rows with gene names.
#                  - Fetches annotations (UniProt, STRING, g:Profiler, QuickGO) for unique selected genes, using caches.
#                  - Maps fetched annotations back to the DataFrame.
#                  - Applies background flags and QC checks.
#                  - Reorders columns for readability.
#                  - Writes the annotated data to a sheet in the output file using `write_sheet_safely`.
#              - Finalizes and saves the output Excel file.
# <<< MODIFIED: Description updated to reflect conditional logic and removal of UniProt GO >>>

def main():
    """
    Main function orchestrating the loading, processing, annotation, and saving.
    Handles sheets with or without a 'Significant' column for filtering.
    Does NOT fetch GO terms directly from UniProt (uses QuickGO instead).
    """
    logging.info(f"--- Starting Main Processing ---")

    # --- Load Input Excel File ---
    # Description: Attempts to load the Excel file specified by INPUT_EXCEL_PATH.
    #              Logs the sheets found or aborts if the file is not found or fails to load.
    try:
        xlsx = pd.ExcelFile(INPUT_EXCEL_PATH)
        sheet_names = xlsx.sheet_names
        logging.info(f"Loaded input Excel: {INPUT_EXCEL_PATH} with sheets: {sheet_names}")
    except FileNotFoundError:
        logging.error(f"CRITICAL: Input file not found at {INPUT_EXCEL_PATH}. Aborting.")
        return
    except Exception as e:
        logging.error(f"CRITICAL: Failed loading input {INPUT_EXCEL_PATH}: {e}. Aborting.")
        return

    # --- Setup Excel Writer ---
    # Description: Initializes the pandas ExcelWriter object used to create the output file.
    #              Creates the output directory if it doesn't exist. Aborts if the writer fails.
    writer = None
    try:
        output_dir = os.path.dirname(output_path)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
            logging.info(f"Created output directory: {output_dir}")
        writer = pd.ExcelWriter(output_path, engine="openpyxl")
        logging.info(f"Initialized Excel writer for {output_path}")
    except Exception as e:
        logging.error(f"CRITICAL: Failed initializing writer for {output_path}: {e}. Aborting.")
        if writer: writer.close()
        return

    # --- Pre-load EV Background Genes (Optional) ---
    # Description: Loads a set of gene symbols from a specified control sheet (defined in CONFIG)
    #              that are considered background contaminants in EV experiments. These genes will be flagged later.
    ev_background_genes: typing.Set[str] = set()
    ev_sheet_name = CONFIG.get("ev_sheet_name")
    if ev_sheet_name:
        if ev_sheet_name in sheet_names:
            try:
                logging.info(f"Loading EV background data from sheet: '{ev_sheet_name}'")
                df_ev_background = xlsx.parse(ev_sheet_name)
                if COL_SIGNIFICANT in df_ev_background.columns and COL_GENE_NAMES in df_ev_background.columns:
                    sig_genes = df_ev_background[df_ev_background[COL_SIGNIFICANT].astype(str).str.strip() == "+"][COL_GENE_NAMES].map(extract_main_gene_symbol)
                    ev_background_genes = {gene for gene in sig_genes if pd.notna(gene)}
                    logging.info(f"Loaded {len(ev_background_genes)} unique background genes from '{ev_sheet_name}'.")
                else:
                    logging.warning(f"EV sheet '{ev_sheet_name}' missing required columns ({COL_SIGNIFICANT}, {COL_GENE_NAMES}). EV Control Background flag cannot be applied.")
            except Exception as e:
                logging.error(f"Error processing EV sheet '{ev_sheet_name}': {e}. EV Control Background flag may be incomplete.")
        else:
            logging.warning(f"Specified EV sheet '{ev_sheet_name}' not found in the input Excel file. EV Control Background flag cannot be applied.")
    else:
        logging.info("No EV background sheet specified (CONFIG['ev_sheet_name']). Skipping EV Control Background flag based on sheet.")


    # --- Define Column Structure Lists (Static Parts) ---
    # Description: Pre-defines lists of column names for different annotation categories.
    #              This helps in organizing the mapping process and the final column order.
    gprofiler_cols = [f"{s} Terms" for s in CONFIG["gprofiler"]["sources"]]
    uniprot_basic_cols = [ COL_UNIPROT_ID, COL_PROTEIN_DESC, COL_FUNCTION, COL_LOCATION, COL_PFAM_DOMAINS, COL_PFAM_DESC, COL_PFAM_CLANS, COL_CANONICAL, COL_MASS_KDA ]
    uniprot_extra_static_cols = [ COL_ALT_NAMES_V2, COL_CAT_ACT_V2, COL_DISEASES_V2, COL_SIMILARITIES_V2, COL_PATHWAYS_V2, COL_EXISTENCE_V2 ]
    uniprot_go_cols = [] # <<< MODIFIED: List is now empty as UniProt GO terms are removed >>>
    quickgo_cols = ["QuickGO:CC Terms", "QuickGO:MF Terms", "QuickGO:BP Terms"]
    string_cols = [COL_STRING_PHYS, COL_STRING_FUNC]
    flag_cols = [COL_STICKY_FLAG, COL_EV_LFQ_FLAG, COL_EV_BG_FLAG, COL_NAME_MATCH]

    all_dynamic_keyword_cols = set()

    # --- Process Each Sheet ---
    # Description: Iterates through each sheet found in the input Excel file.
    processed_sheet_count = 0
    total_sheets = len(sheet_names)

    for i, sheet_name in enumerate(sheet_names):
        logging.info(f"--- Processing sheet {i+1}/{total_sheets}: '{sheet_name}' ---")
        df = None; df_sig = None # Initialize sheet DataFrames for this iteration.
        try:
            # --- Load Sheet Data ---
            df = xlsx.parse(sheet_name)
            logging.info(f"Loaded sheet '{sheet_name}' ({df.shape[0]} rows x {df.shape[1]} columns)")

            # --- Check for Essential 'Gene names' Column ---
            if COL_GENE_NAMES not in df.columns:
                logging.warning(f"Sheet '{sheet_name}' missing essential column '{COL_GENE_NAMES}'. Cannot process. Skipping annotation.")
                write_sheet_safely(writer, sheet_name, df)
                processed_sheet_count += 1
                continue

            # --- Determine Rows to Annotate (Conditional Logic) ---
            if COL_SIGNIFICANT in df.columns:
                # Mode 1: 'Significant' column EXISTS. Filter based on its content.
                logging.info(f"Found '{COL_SIGNIFICANT}' column. Filtering for significant rows ('+').")
                try:
                    df[COL_SIGNIFICANT] = df[COL_SIGNIFICANT].astype(str)
                    df_sig = df[df[COL_SIGNIFICANT].str.strip() == "+"].copy()
                    logging.info(f"Found {df_sig.shape[0]} significant ('+') rows to process.")
                except Exception as e:
                    logging.error(f"Error processing '{COL_SIGNIFICANT}' column in sheet '{sheet_name}': {e}. Skipping annotation for this sheet.")
                    write_sheet_safely(writer, sheet_name, df); processed_sheet_count += 1; continue
            else:
                # Mode 2: 'Significant' column is MISSING. Annotate all rows with valid gene names.
                logging.info(f"'{COL_SIGNIFICANT}' column not found in sheet '{sheet_name}'. Processing all rows with entries in '{COL_GENE_NAMES}'.")
                df_sig = df.dropna(subset=[COL_GENE_NAMES]).copy()
                df_sig = df_sig[df_sig[COL_GENE_NAMES].astype(str).str.strip() != '']
                logging.info(f"Selected {df_sig.shape[0]} rows with gene names to process.")

            # --- Handle Empty Selection ---
            if df_sig.empty:
                logging.info(f"No rows selected for annotation in sheet '{sheet_name}'. Writing empty annotated sheet.")
                empty_cols = list(df.columns)
                # <<< MODIFIED: uniprot_go_cols is empty, so effectively removed >>>
                generated_cols_order = (
                     [COL_MAIN_GENE] + uniprot_basic_cols + uniprot_extra_static_cols +
                     uniprot_go_cols + quickgo_cols +
                     string_cols + gprofiler_cols + flag_cols
                )
                for col in generated_cols_order:
                    if col not in empty_cols: empty_cols.append(col)
                empty_df_sig = pd.DataFrame(columns=empty_cols)
                write_sheet_safely(writer, sheet_name, empty_df_sig)
                processed_sheet_count += 1
                continue

            # --- Extract Main Gene Symbol ---
            try:
                df_sig[COL_MAIN_GENE] = df_sig[COL_GENE_NAMES].apply(extract_main_gene_symbol)
            except Exception as e:
                logging.error(f"Error extracting main gene symbol in sheet '{sheet_name}': {e}. '{COL_MAIN_GENE}' column will have NAs.")
                df_sig[COL_MAIN_GENE] = pd.NA

            # --- Identify Unique Genes for Annotation ---
            unique_genes = df_sig[COL_MAIN_GENE].dropna().unique().tolist()
            logging.info(f"Annotating {len(unique_genes)} unique valid gene symbols for '{sheet_name}'...")

            # --- Fetch Annotations Loop ---
            sheet_dynamic_keyword_cols = set()
            if not unique_genes:
                logging.info("No valid unique gene symbols found to annotate.")
            else:
                for gene in tqdm(unique_genes, desc=f"Annotating '{sheet_name}'", unit="gene", ncols=100, leave=False):
                    if not gene: continue

                    # Fetch/Process UniProt data (excluding UniProt GO)
                    uniprot_entry = fetch_uniprot_entry(gene)
                    process_uniprot_basic_fields(gene, uniprot_entry)
                    extra_result = process_uniprot_extra_fields(gene, uniprot_entry) # <<< No longer gets UniProt GO >>>
                    if extra_result:
                        for key in extra_result.keys():
                            if isinstance(key, str) and key.startswith("UniProt KW:"):
                                sheet_dynamic_keyword_cols.add(key)

                    # Fetch g:Profiler terms
                    fetch_gprofiler_enrichment(gene)
                    # time.sleep(...) removed

                    # Fetch STRING interactors
                    fetch_string_interactors(gene, network_type="physical")
                    fetch_string_interactors(gene, network_type="functional")

                    # Fetch QuickGO terms
                    fetch_quickgo_annotations(gene)

                logging.info(f"Finished fetching annotations for '{sheet_name}'.")
                all_dynamic_keyword_cols.update(sheet_dynamic_keyword_cols)
                if sheet_dynamic_keyword_cols:
                    logging.info(f"Dynamic UniProt Keyword columns found for this sheet: {sorted(list(sheet_dynamic_keyword_cols))}")

            # --- Define FULL Annotation Column List for Mapping ---
            current_dynamic_keyword_cols_sorted = sorted(list(sheet_dynamic_keyword_cols))
            # <<< MODIFIED: uniprot_go_cols is empty, effectively removed >>>
            all_annotation_cols_for_mapping = (
                uniprot_basic_cols +
                uniprot_extra_static_cols +
                # uniprot_go_cols + # This list is empty
                quickgo_cols +
                current_dynamic_keyword_cols_sorted +
                string_cols +
                gprofiler_cols
            )

            # --- Map Annotations to DataFrame ---
            logging.info(f"Mapping {len(all_annotation_cols_for_mapping)} annotation columns back to the sheet...")
            for col in all_annotation_cols_for_mapping:
                if col not in df_sig.columns: df_sig[col] = pd.NA
                try:
                    gene_map_col_upper = df_sig[COL_MAIN_GENE].str.upper()
                    if col in uniprot_basic_cols:
                        cache = global_cache["uniprot_basic"]
                        df_sig[col] = gene_map_col_upper.map(lambda g: cache.get(g, {}).get(col, pd.NA) if pd.notna(g) else pd.NA)
                    # <<< MODIFIED: Removed explicit check for COL_UNIPROT_GO_CC >>>
                    elif col in uniprot_extra_static_cols or (isinstance(col, str) and col.startswith("UniProt KW:")):
                        cache = global_cache["uniprot_extra"]
                        df_sig[col] = gene_map_col_upper.map(lambda g: cache.get(g, {}).get(col, pd.NA) if pd.notna(g) else pd.NA)
                    elif col == COL_STRING_PHYS:
                        cache = global_cache["string_physical"]
                        df_sig[col] = gene_map_col_upper.map(cache)
                    elif col == COL_STRING_FUNC:
                        cache = global_cache["string_functional"]
                        df_sig[col] = gene_map_col_upper.map(cache)
                    elif col in gprofiler_cols:
                        cache = global_cache["gprofiler"]
                        df_sig[col] = gene_map_col_upper.map(lambda g: cache.get(g, {}).get(col, pd.NA) if pd.notna(g) else pd.NA)
                    elif col == "QuickGO:CC Terms":
                        cache = global_cache["quickgo_cc"]
                        df_sig[col] = gene_map_col_upper.map(cache)
                    elif col == "QuickGO:MF Terms":
                        cache = global_cache["quickgo_mf"]
                        df_sig[col] = gene_map_col_upper.map(cache)
                    elif col == "QuickGO:BP Terms":
                        cache = global_cache["quickgo_bp"]
                        df_sig[col] = gene_map_col_upper.map(cache)
                except Exception as map_e:
                    logging.error(f"Error mapping column '{col}' in sheet '{sheet_name}': {map_e}. Filling with NA.")
                    df_sig[col] = pd.NA

            # --- Apply Flags and QC Checks ---
            logging.info("Applying flags and QC checks...")
            if COL_PROTEIN_DESC in df_sig.columns:
                df_sig[COL_STICKY_FLAG] = df_sig[COL_PROTEIN_DESC].apply(detect_sticky_binder)
            else:
                logging.warning(f"Cannot apply Sticky Flag: Column '{COL_PROTEIN_DESC}' missing in sheet '{sheet_name}'.")
                df_sig[COL_STICKY_FLAG] = ""
            if set(CONFIG["ev_columns"]).issubset(df_sig.columns) and COL_LOG2FC in df_sig.columns:
                df_sig[COL_EV_LFQ_FLAG] = df_sig.apply(detect_ev_lfq_background, axis=1)
            else:
                 logging.warning(f"Skipping EV LFQ Background flag in sheet '{sheet_name}': Required columns missing (e.g., {CONFIG['ev_columns']}, {COL_LOG2FC}).")
                 df_sig[COL_EV_LFQ_FLAG] = ""
            if ev_background_genes:
                df_sig[COL_EV_BG_FLAG] = df_sig[COL_MAIN_GENE].apply(lambda g: "EV Control Background" if pd.notna(g) and g in ev_background_genes else "")
                logging.info("Applied EV Control Background flag.")
            else:
                df_sig[COL_EV_BG_FLAG] = ""
            if COL_PROTEIN_NAMES in df_sig.columns and COL_PROTEIN_DESC in df_sig.columns:
                 df_sig[COL_NAME_MATCH] = df_sig.apply(lambda r: "" if pd.isna(r[COL_PROTEIN_NAMES]) or pd.isna(r[COL_PROTEIN_DESC]) else ("" if str(r[COL_PROTEIN_NAMES]).strip().lower() in str(r[COL_PROTEIN_DESC]).lower() else f"Mismatch: Input='{str(r[COL_PROTEIN_NAMES])[:50]}' vs UniProt='{str(r[COL_PROTEIN_DESC])[:50]}'"), axis=1)
            else:
                 logging.warning(f"Skipping Name Match QC in sheet '{sheet_name}': Columns '{COL_PROTEIN_NAMES}' or '{COL_PROTEIN_DESC}' missing.")
                 df_sig[COL_NAME_MATCH] = ""

            # --- Reorder Columns for Final Output ---
            # Description: Arranges the columns in the `df_sig` DataFrame into a logical order,
            #              using the corrected logic to insert annotations after 'Gene names'.
            logging.info("Reordering columns for final output...")
            try:
                # <<< Using Corrected Reordering Logic >>>
                # Define the desired order of the newly generated/added columns.
                # <<< MODIFIED: uniprot_go_cols is empty, effectively removed >>>
                ordered_generated_cols = (
                    [COL_MAIN_GENE] + uniprot_basic_cols + uniprot_extra_static_cols +
                    # uniprot_go_cols + # This list is empty
                    quickgo_cols + current_dynamic_keyword_cols_sorted +
                    string_cols + gprofiler_cols + flag_cols
                )
                # Filter this list to include only columns actually present in the current df_sig.
                ordered_generated_cols_present = [c for c in ordered_generated_cols if c in df_sig.columns]

                # Determine the split point in the *original* columns list (`df.columns`)
                original_cols = df.columns.tolist()
                cols_before_insertion = []
                cols_after_insertion = []

                if COL_GENE_NAMES in original_cols:
                    try:
                        insert_idx = original_cols.index(COL_GENE_NAMES)
                        cols_before_insertion = original_cols[:insert_idx + 1]
                        cols_after_insertion = original_cols[insert_idx + 1:]
                        logging.debug(f"Reordering: Found '{COL_GENE_NAMES}' at index {insert_idx}.")
                    except ValueError:
                        logging.warning(f"'{COL_GENE_NAMES}' not found in original cols index? Using all original columns first.")
                        cols_before_insertion = original_cols
                else:
                    logging.warning(f"'{COL_GENE_NAMES}' not found in original columns. Placing generated columns after all originals.")
                    cols_before_insertion = original_cols

                # Construct the final column order carefully.
                final_col_order = []
                seen_cols = set()

                # 1. Add original columns up to the insertion point (if they exist in df_sig).
                for col in cols_before_insertion:
                    if col in df_sig.columns and col not in seen_cols:
                        final_col_order.append(col)
                        seen_cols.add(col)

                # 2. Add the generated annotation columns (if they exist in df_sig and haven't been added).
                for col in ordered_generated_cols_present:
                    if col in df_sig.columns and col not in seen_cols:
                        final_col_order.append(col)
                        seen_cols.add(col)

                # 3. Add the remaining original columns (after insertion point) (if they exist in df_sig and haven't been added).
                for col in cols_after_insertion:
                     if col in df_sig.columns and col not in seen_cols:
                        final_col_order.append(col)
                        seen_cols.add(col)

                # 4. Safety Net: Add any columns from df_sig that were missed entirely.
                for col in df_sig.columns:
                     if col not in seen_cols:
                         logging.warning(f"Column '{col}' appended at end during reordering (was missed from logic).")
                         final_col_order.append(col)

                # Apply the reordered list to the DataFrame.
                df_sig = df_sig[final_col_order]
                logging.info(f"Successfully reordered columns. Final shape for sheet '{sheet_name}': ({df_sig.shape[0]}x{df_sig.shape[1]})")

            except Exception as e:
                logging.error(f"Column reordering failed for sheet '{sheet_name}': {e}. Using default order before saving.")


            # --- Save Processed Sheet ---
            write_sheet_safely(writer, sheet_name, df_sig)
            processed_sheet_count += 1

        except Exception as e:
            # --- Fatal Error Handling for a Specific Sheet ---
            logging.exception(f"FATAL ERROR processing sheet '{sheet_name}': {e}")
            logging.info(f"Attempting to write original data (or empty sheet) for errored sheet '{sheet_name}'.")
            try:
                if 'df' in locals() and df is not None:
                    write_sheet_safely(writer, sheet_name, df)
                    processed_sheet_count += 1
                else:
                    logging.error(f"Original df not available for errored sheet '{sheet_name}'. Writing empty DataFrame.")
                    write_sheet_safely(writer, sheet_name, pd.DataFrame())
                    processed_sheet_count += 1
            except Exception as nested_e:
                logging.error(f"Could not write original or empty data for errored sheet '{sheet_name}': {nested_e}")

    # === 14. FINALIZE EXCEL FILE ===
    # Description: Closes the ExcelWriter, which triggers the final save of the output file.
    #              Logs summary statistics about the run.
    try:
        if writer and hasattr(writer, 'book') and writer.book is not None:
             writer.close() # Closing the writer saves the Excel file.
             logging.info(f"--- Main Processing Finished ---")
             logging.info(f"✅ Annotation and saving complete.")
             logging.info(f"   Processed/Written Sheets: {processed_sheet_count}/{total_sheets}")
             logging.info(f"   Output saved at: {output_path}")
             logging.info(f"   Log file saved at: {log_file_name}")
             if all_dynamic_keyword_cols:
                 logging.info(f"   Unique dynamic UniProt Keyword columns generated across all sheets: {sorted(list(all_dynamic_keyword_cols))}")
        elif writer:
            logging.warning("Excel writer object existed but seemed invalid for final closing/saving. Output might be incomplete.")
        else:
            logging.error("Excel writer object was not created successfully. No output file generated.")
    except Exception as e:
        logging.error(f"CRITICAL: Failed to close or finalize the Excel writer: {e}")

    logging.info(f"--- Script Execution Finished ---")

# === 15. SCRIPT EXECUTION POINT ===
# Description: This block ensures that the `main` function is called only when the script
#              is executed directly (not imported as a module). It also records and logs
#              the total execution time. Includes commented-out code for clearing caches.
if __name__ == "__main__":
    start_time = time.time() # Record the start time.

    # --- Optional: Clear Caches for Fresh Run ---
    # Description: Useful in interactive environments (like Jupyter) to force refetching
    #              of all API data on every run, ignoring previously cached results.
    # global_cache = {
    #  "uniprot_entry": {}, "uniprot_basic": {}, "uniprot_extra": {},
    #  "gprofiler": {}, "string_physical": {}, "string_functional": {},
    #  "quickgo_cc": {}, "quickgo_mf": {}, "quickgo_bp": {}
    # }
    # logging.info("Global caches cleared for new run.")
    # ---------------------------------------------

    # Call the main processing function defined above.
    main()

    end_time = time.time() # Record the end time.
    duration = end_time - start_time # Calculate the duration.
    # Log the total time taken for the script execution.
    logging.info(f"Total script execution time: {duration:.2f} seconds.")

2025-04-29 11:54:17 [INFO    ] --- Script Execution Started ---
2025-04-29 11:54:17 [INFO    ] Input file: /Users/umuttank/Downloads/Results.xlsx
2025-04-29 11:54:17 [INFO    ] Output file: /Users/umuttank/Downloads/Results.xlsx
2025-04-29 11:54:17 [INFO    ] Pfam Clan file: Pfam-A.clans.tsv.gz
2025-04-29 11:54:17 [INFO    ] Log file: annotation_log_20250429.log
2025-04-29 11:54:17 [INFO    ] Requests session configured with max 5 retries and backoff factor 0.4.
2025-04-29 11:54:17 [INFO    ] Successfully loaded Pfam clan data (24424 rows) from Pfam-A.clans.tsv.gz
2025-04-29 11:54:17 [INFO    ] Initialized global caches for API results.
2025-04-29 11:54:17 [INFO    ] g:Profiler client initialized.
2025-04-29 11:54:17 [INFO    ] --- Starting Main Processing ---
2025-04-29 11:54:17 [INFO    ] Loaded input Excel: /Users/umuttank/Downloads/ladin test.xlsx with sheets: ['Sheet1']
2025-04-29 11:54:17 [INFO    ] Initialized Excel writer for /Users/umuttank/Downloads/ladin test_Annotated_20250